In [2]:
#| default_exp cli

# Command Line Tools

In [3]:
#| export
from fastcore.utils import *
from fastcore.script import call_parse, bool_arg
from subprocess import check_output, run
import sys
import uvicorn
from pprint import pprint
import json
import toml

In [4]:
#| export
@call_parse
def railway_link():
    "Link the current directory to the current project's Railway service"
    j = json.loads(check_output("railway status --json".split()))
    prj = j['id']
    idxpath = 'edges', 0, 'node', 'id'
    env = nested_idx(j, 'environments', *idxpath)
    svc = nested_idx(j, 'services', *idxpath)

    cmd = f"railway link -e {env} -p {prj} -s {svc}"
    res = check_output(cmd.split())

In [5]:
#| export
def _run(a, **kw):
    print('#', ' '.join(a))
    run(a)

In [6]:
#| export
@call_parse
def railway_deploy(
    name:str, # The project name to deploy
    mount:bool_arg=True # Create a mounted volume at /app/data?
):
    """Deploy a FastHTML app to Railway"""
    nm,ver = check_output("railway --version".split()).decode().split()
    assert nm=='railwayapp', f'Unexpected railway version string: {nm}'
    if ver2tuple(ver)<(3,8): return print("Please update your railway CLI version to 3.8 or higher")
    cp = run("railway status --json".split(), capture_output=True)
    if not cp.returncode:
        print("Checking deployed projects...")
        project_name = json.loads(cp.stdout.decode()).get('name')
        if project_name == name: return print("This project is already deployed. Run `railway open`.")
    reqs = Path('requirements.txt')
    if not reqs.exists(): reqs.write_text('python-fasthtml')
    _run(f"railway init -n {name}".split())
    _run(f"railway up -c".split())
    _run(f"railway domain".split())
    railway_link.__wrapped__()
    if mount: _run(f"railway volume add -m /app/data".split())
    _run(f"railway up -c".split())

In [7]:
#| export
def default_cfg():
    return """
app='app'

[dev]  # A dev stage
host='0.0.0.0'
port=5001
reload=true

# reload_includes=[]  # Additional files to watch for changes
# reload_excludes=[]  # Files to ignore for changes

# Add scripts to be run
# scripts = []  # Scripts / commands to run before starting the app

# [build]  # Uncomment to add a build stage

[prod]
host='0.0.0.0'
port=5001
reload=false
# scripts = []  # Scripts / commands to run before starting the app
"""

In [8]:
#| export
@call_parse
def fasthtml(
    mode:str, # dev, prod or custom mode (in fasthtml.toml)
    appname:str # Name of the module and app instance to be served. e.g. main.py:app
):
    """Run FastHTML app in a specific mode."""
    cfg = toml.loads(default_cfg())
    cfg_path = Path('fasthtml.toml')
    if cfg_path.exists():
        for sec, val in toml.load(cfg_path).items():
            s=cfg.get(sec)
            if s and isinstance(s, dict):
                s.update(val)
            else:
                cfg[sec] = val
        #cfg.update(toml.load(cfg_path))
    section = cfg.get(mode)
    if not section: print(f'section {mode} does not exist! Use dev, prod or add a custom section in fasthtml.toml')
    if commands:=section.get('scripts'):
        for command in commands:
            os.system(command)
    pprint(cfg)
    if section.get('start', True):
        print(str(Path().resolve()))
        sys.path.append(str(Path().resolve()))
        host = section.get('host', 'http://localhost')
        port = int(section.get('port', 5001))
        reload = section.get('reload', True)
        reload_includes = section.get('reload_includes', None)
        reload_excludes = section.get('reload_excludes', None)
        print(f'Starting {appname}')
        print(f'Link: {host}:{port}')
        uvicorn.run(f'{appname}', host=host, port=port, reload=reload, reload_includes=reload_includes, reload_excludes=reload_excludes)

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()